In [1]:
%%configure -f
{
    "conf":
    {
        "spark.driver.extraJavaOptions" : "--add-opens java.base/jdk.internal.loader=ALL-UNNAMED",
        "spark.jars.packages": "com.google.protobuf:protobuf-java:3.25.5"
    }
}

StatementMeta(, b332cd77-e92d-41ba-8294-3ab6101ad7c1, -1, Finished, Available, Finished)

In [2]:
# ESRI - FABRIC reference: https://developers.arcgis.com/geoanalytics-fabric/

# Import the required modules
import geoanalytics_fabric
from geoanalytics_fabric.sql import functions as ST
from geoanalytics_fabric import extensions

StatementMeta(, b332cd77-e92d-41ba-8294-3ab6101ad7c1, 5, Finished, Available, Finished)

In [3]:
# Read ahn file from OneLake
# AHN lidar data source: https://viewer.ahn.nl/

ahn_csv_path = "Files/AHN lidar/AHN4_csv"
lidar_df = spark.read.options(delimiter=" ").csv(ahn_csv_path)
lidar_df = lidar_df.selectExpr("_c0 as X", "_c1 as Y", "_c2 Z")

lidar_df.printSchema()
lidar_df.show(5)
lidar_df.count()

StatementMeta(, b332cd77-e92d-41ba-8294-3ab6101ad7c1, 6, Finished, Available, Finished)

root
 |-- X: string (nullable = true)
 |-- Y: string (nullable = true)
 |-- Z: string (nullable = true)

+----------+----------+-----+
|         X|         Y|    Z|
+----------+----------+-----+
|240000.060|594999.650|0.294|
|240000.053|594997.798|0.306|
|240000.031|594996.160|0.282|
|240000.020|594993.946|0.304|
|240000.077|594992.847|0.319|
+----------+----------+-----+
only showing top 5 rows



2184749356

In [4]:
# Create point geometry from x,y,z columns and set the spatial refrence system
lidar_df = lidar_df.select(ST.make_point(x="X", y="Y", z="Z").alias("rd_point"))
lidar_df = lidar_df.withColumn("srid", ST.srid("rd_point"))
lidar_df = lidar_df.select(ST.srid("rd_point", 28992).alias("rd_point"))\
  .withColumn("srid", ST.srid("rd_point"))

lidar_df.printSchema()
lidar_df.show(5)

StatementMeta(, b332cd77-e92d-41ba-8294-3ab6101ad7c1, 7, Finished, Available, Finished)

root
 |-- rd_point: point (nullable = true)
 |-- srid: integer (nullable = true)

+--------------------+-----+
|            rd_point| srid|
+--------------------+-----+
|{"x":240000.06,"y...|28992|
|{"x":240000.053,"...|28992|
|{"x":240000.031,"...|28992|
|{"x":240000.02,"y...|28992|
|{"x":240000.077,"...|28992|
+--------------------+-----+
only showing top 5 rows



In [5]:
results_path = "Files/AHN lidar/AHN4_parquet/ahn4.parquet"
lidar_df.write.save(results_path, format="geoparquet")

StatementMeta(, b332cd77-e92d-41ba-8294-3ab6101ad7c1, 8, Finished, Available, Finished)

In [21]:
lidar_df.count()

StatementMeta(, 695df883-5021-4cc8-bbec-3070ddcfa4a4, 24, Finished, Available, Finished)

1473917112